# Training own eng_pol mT5 translator 

input data was prepared in another notbook https://github.com/len-sla/NLP_mT5_translation/blob/main/preparing_input_for_mT5.ipynb

---

Notbeook is based on the following sources:
https://towardsdatascience.com/how-to-train-an-mt5-model-for-translation-with-simple-transformers-30ba5fa66c5f
https://simpletransformers.ai/docs/tips-and-tricks/#saving-checkpoints

## 1. Mounting google drive and loading previously prepared input csv files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

In [ ]:
train_df= pd.read_csv("/content/drive/MyDrive/mt5/data/eng-pol/train.tsv", sep="\t")
eval_df = pd.read_csv("/content/drive/MyDrive/mt5/data/eng-pol/eval.tsv", sep="\t")

In [ ]:
def change_df_col_toString(df):
  for i in df.columns:
    df[i] = df[i].astype('str')
  return df

In [ ]:
train_df=change_df_col_toString(train_df)
eval_df=change_df_col_toString(eval_df)

In [ ]:
train_df.sample(10)

,Unnamed: 0,prefix,input_text,target_text
43413,43413,translate english to polish,- My lord!,Okropny sen miałem.
504114,504114,translate polish to english,"Sprawozdawcą był André MORDANT, a współsprawoz...",The rapporteur was Mr Mordant and the co-rappo...
1548314,1548314,translate polish to english,{y: i}Mój plan,Come
250387,250387,translate english to polish,We won! Everyone...,Ludzie...
1081121,1081121,translate english to polish,"- Hi, Mrs. Baskets.",- Witam panią.
1359412,1359412,translate polish to english,Nie wolno przerywać przyjmowania insuliny bez ...,Do not stop taking your insulin without speaki...
1896996,1896996,translate polish to english,Jest czymś więcej.,She's more than that.
1893220,1893220,translate polish to english,To buduje charakter.,It builds character.
1043082,1043082,translate polish to english,Czego chcesz?,What is it?
1715913,1715913,translate english to polish,Article 149,Artykuł 149


In [ ]:
eval_df.sample(10)

,Unnamed: 0,prefix,input_text,target_text
7184,7184,translate polish to english,Jadę do Bostonu odwiedzić Toma.,I'm going to Boston to visit Tom.
19903,19903,translate english to polish,"For sale: baby shoes, never worn.","Na sprzedaż: butki dziecięce, nieużywane."
4979,4979,translate english to polish,I never knew you had children.,"Nie wiedziałem, że miałeś dzieci."
8059,8059,translate english to polish,I can't get through to her.,Nie mogę się do niej dodzwonić.
12808,12808,translate polish to english,"Powiedzieli mi, że są bardzo zmęczeni.",They told me they were very tired.
2857,2857,translate english to polish,You are a prisoner.,Jesteś więźniem.
17179,17179,translate english to polish,You are beautiful.,Jesteś piękna.
8387,8387,translate english to polish,Tom's adopted.,Tom jest adoptowany.
2829,2829,translate english to polish,Help me!,Pomóż mi!
2116,2116,translate polish to english,Jestem nieśmiały.,I'm shy.


In case some extra column appears kill it

In [ ]:
def kill_col(df):
        if 'Unnamed: 0' in df:
            del df['Unnamed: 0']


            
        return df

In [ ]:
eval_df= kill_col(eval_df)
train_df= kill_col(train_df)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   prefix       object
 1   input_text   object
 2   target_text  object
dtypes: object(3)
memory usage: 45.8+ MB


In [ ]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prefix       20000 non-null  object
 1   input_text   20000 non-null  object
 2   target_text  20000 non-null  object
dtypes: object(3)
memory usage: 468.9+ KB


there were strange errors suggesting that column is not str type so I forced that above

In [ ]:
train_df.shape, eval_df.shape

((2000000, 3), (20000, 3))

Column prefix was killed as even without model should coupe with the training task 

In [ ]:
train_df["prefix"] = ""
eval_df["prefix"] = ""

## 2.Loading transformers library for mT5

https://github.com/google-research/multilingual-t5

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 4.4MB/s 
     |████████████████████████████████| 225kB 42.7MB/s 
     |████████████████████████████████| 8.2MB 39.6MB/s 
     |████████████████████████████████| 3.3MB 33.5MB/s 
     |████████████████████████████████| 2.1MB 32.8MB/s 
     |████████████████████████████████| 81kB 347kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 2.1MB 31.7MB/s 
     |████████████████████████████████| 1.2MB 28.0MB/s 
     |████████████████████████████████| 122kB 27.7MB/s 
     |████████████████████████████████| 245kB 40.3MB/s 
     |████████████████████████████████| 112kB 30.8MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 163kB 43.0MB/s 
     |████████████████████████████████| 4.2MB 33.3MB/s 
     |████████████████████████████████| 112kB 45.9MB/s 
     |████████████████████████████████| 901kB 31.8MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 


In [ ]:
import logging
# import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

default pameters

In [ ]:
# model_args = T5Args()
# model_args.max_seq_length = 96
# model_args.train_batch_size = 20
# model_args.eval_batch_size = 20
# model_args.num_train_epochs = 1
# model_args.evaluate_during_training = True
# model_args.evaluate_during_training_steps = 30000
# model_args.use_multiprocessing = False
# model_args.fp16 = False
# model_args.save_steps = -1
# model_args.save_eval_checkpoints = False
# model_args.no_cache = True
# model_args.reprocess_input_data = True
# model_args.overwrite_output_dir = True
# model_args.preprocess_inputs = False
# model_args.num_return_sequences = 1
# model_args.wandb_project = "MT5 Sinhala-English Translation"

# model = T5Model("mt5", "google/mt5-base", args=model_args)

current session parameters


In [ ]:
model_args = T5Args()
model_args.max_seq_length = 32
model_args.train_batch_size = 4 
model_args.eval_batch_size = 4
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 10000
model_args.use_multiprocessing = False
model_args.fp16 = True
model_args.save_steps = True
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Polish-English Translation"

model = T5Model("mt5", "google/mt5-base", args=model_args)

INFO:filelock:Lock 139699571937744 acquired on /root/.cache/huggingface/transformers/5ebfd830555547194403d6803baa127970de59b443c04b7a1a60b16a97ed3958.3950cd4aaa701cb6f55a976ff996001a5fb09bbbe7ba9084619949d9016f519e.lock


Downloading:   0%|          | 0.00/639 [00:00<?, ?B/s]

INFO:filelock:Lock 139699571937744 released on /root/.cache/huggingface/transformers/5ebfd830555547194403d6803baa127970de59b443c04b7a1a60b16a97ed3958.3950cd4aaa701cb6f55a976ff996001a5fb09bbbe7ba9084619949d9016f519e.lock
INFO:filelock:Lock 139699564690064 acquired on /root/.cache/huggingface/transformers/3b7e8056d4ed71d8d7ac2dea78627c4be77ed136399c05b563d4116abfcd9418.1afec9001b62cd5a347e7fd4b664e503ca2377606e11b9ddb8ec1d7b79bc3952.lock


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

INFO:filelock:Lock 139699564690064 released on /root/.cache/huggingface/transformers/3b7e8056d4ed71d8d7ac2dea78627c4be77ed136399c05b563d4116abfcd9418.1afec9001b62cd5a347e7fd4b664e503ca2377606e11b9ddb8ec1d7b79bc3952.lock
INFO:filelock:Lock 139699556220752 acquired on /root/.cache/huggingface/transformers/4764ec347af4d2d6286acbe1d9d630ac0afd8554a4c4a64170e0b663fd2e2412.84ea7af2df68dc8db434d3160aab65cce8ac63ce5b6f7743f8c9a4a14b4f77e2.lock


...

INFO:filelock:Lock 139699556220752 released on /root/.cache/huggingface/transformers/4764ec347af4d2d6286acbe1d9d630ac0afd8554a4c4a64170e0b663fd2e2412.84ea7af2df68dc8db434d3160aab65cce8ac63ce5b6f7743f8c9a4a14b4f77e2.lock
INFO:filelock:Lock 139701964057936 acquired on /root/.cache/huggingface/transformers/0d7d5b3fc19bf58d4b274990c8bcf5e307726bc18d95f40a1436dfb6a0892f85.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82.lock


Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

INFO:filelock:Lock 139701964057936 released on /root/.cache/huggingface/transformers/0d7d5b3fc19bf58d4b274990c8bcf5e307726bc18d95f40a1436dfb6a0892f85.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82.lock
INFO:filelock:Lock 139701963455504 acquired on /root/.cache/huggingface/transformers/afba33be693521ccefbde6d03b93b5c517d7108ba31f6c08000ed52c2cea45c9.28bbf90ae7962b1b7211c0ce8b2006f968c82439ec9c47e0847ba63642f9435a.lock


Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

INFO:filelock:Lock 139701963455504 released on /root/.cache/huggingface/transformers/afba33be693521ccefbde6d03b93b5c517d7108ba31f6c08000ed52c2cea45c9.28bbf90ae7962b1b7211c0ce8b2006f968c82439ec9c47e0847ba63642f9435a.lock


## 3. Training model

3.1. First attempt with default parameters:

In [ ]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/2000000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


3.2. Second attempt
with batch size 20 trainig crashed

In [ ]:
# second attempt Train the model with batc size 10
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/2000000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


3.3. Seventh attempt

This attempt crashed but this time not because of parameters for the traing but available space for free of charge Colab space :))

In [ ]:
# second attempt Train the model with batc size 4, nr of steps 10000
model.train_model(train_df, eval_data=eval_df)

Output of the cell was edited  to remove login  in point
https://wandb.ai/xxxxxx/





```
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/

100%
2000000/2000000 [23:03<00:00, 1474.54it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
INFO:simpletransformers.t5.t5_model: Training started

Using Adafactor for T5

Epoch 1 of 1: 0%
0/1 [12:51<?, ?it/s]

wandb: You can find your API key in your browser here: https://wandb.ai/authorize

wandb: Paste an API key from your profile and hit enter: ··········

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc

Tracking run with wandb version 0.10.29
Syncing run morning-silence-3 to Weights & Biases (Documentation).
Project page: https://wandb.ai/xxxxxx/MT5%20Polish-English%20Translation
Run page: https://wandb.ai/xxxxxx/MT5%20Polish-English%20Translation/runs/fflkc6mv
Run data is saved locally in /content/wandb/run-20210505_125522-fflkc6mv

Epochs 0/1. Running Loss: nan: 0%
12/500000 [11:46<8054:17:39, 57.99s/it]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Exception in thread Thread-17:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 172, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 747, in communicate_network_status
    resp = self._communicate(req, timeout=timeout, local=True)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 537, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 542, in _communicate_async
    raise Exception("The wandb backend process has shutdown")
Exception: The wandb backend process has shutdown

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 190, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 735, in communicate_stop_status
    resp = self._communicate(req, timeout=timeout, local=True)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 537, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 542, in _communicate_async
    raise Exception("The wandb backend process has shutdown")
Exception: The wandb backend process has shutdown

---------------------------------------------------------------------------

OSError                                   Traceback (most recent call last)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py in save(obj, f, pickle_module, pickle_protocol, _use_new_zipfile_serialization)
    371             with _open_zipfile_writer(opened_file) as opened_zipfile:
--> 372                 _save(obj, opened_zipfile, pickle_module, pickle_protocol)
    373                 return

7 frames

OSError: [Errno 28] No space left on device


During handling of the above exception, another exception occurred:

RuntimeError                              Traceback (most recent call last)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py in __exit__(self, *args)
    257 
    258     def __exit__(self, *args) -> None:
--> 259         self.file_like.write_end_of_file()
    260         self.buffer.flush()
    261 

RuntimeError: [enforce fail at inline_container.cc:274] . unexpected pos 1265525504 vs 1265525392

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f0e06888550>> (for post_run_cell):

---------------------------------------------------------------------------

Exception                                 Traceback (most recent call last)

/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py in _pause_backend(self)
    252                 res = self.run.log_code(root=None)
    253                 logger.info("saved code: %s", res)
--> 254             self.backend.interface.publish_pause()
    255 
    256     def _resume_backend(self):

1 frames

/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py in _publish(self, record, local)
    526     def _publish(self, record: pb.Record, local: bool = None) -> None:
    527         if self._process and not self._process.is_alive():
--> 528             raise Exception("The wandb backend process has shutdown")
    529         if local:
    530             record.control.local = local

Exception: The wandb backend process has shutdown
```



